In [97]:
import json 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

# Valid data parsing first
with open('./samples/valid/aircraft_valid.json') as json_file:
    data_valid = json.load(json_file)
print(data_valid['aircraft'][0])
print("Valid count: ", len(data_valid['aircraft']))
# Parse spoofed data
with open('./samples/spoofed/aircraft_spoofed_large.json') as json_file:
    data_spoofed = json.load(json_file)
    
aircraft_data_valid = data_valid['aircraft']
aircraft_data_spoofed = data_spoofed['aircraft']

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

print("Datasets imported!")

{'hex': '845f9f', 'type': 'adsc', 'flight': 'KZ51    ', 'r': 'JA11KZ', 't': 'B748', 'alt_baro': 31996, 'gs': 487.0, 'track': 244.0, 'baro_rate': 48, 'lat': 57.32872, 'lon': -177.562752, 'nic': 0, 'rc': 0, 'seen_pos': 873.399, 'mlat': [], 'tisb': [], 'messages': 4420465, 'seen': 803.5, 'rssi': -29.2}
Valid count:  8423
Datasets imported!


In [98]:
# Define a function to preprocess the data
def preprocess_data(data):
    features = []
    labels = []

    for aircraft in data:
        try:
            feature_vector = [
                #alt_baro maybe float?
                handle_alt_baro(aircraft.get('alt_baro', 0)),
                float(aircraft.get('gs', 0)),
                float(aircraft.get('track', 0)),
                float(aircraft.get('baro_rate', 0)),
                float(aircraft.get('lat', 0)),
                float(aircraft.get('lon', 0)),
                float(aircraft.get('seen_pos', 0)),
                float(aircraft.get('messages', 0)),
                float(aircraft.get('seen', 0)),
                float(aircraft.get('rssi', 0)),
            ]
            features.append(feature_vector)
            labels.append(1 if aircraft.get('is_spoofed', False) else 0)
        except ValueError as e:
            # Handle the case where a non-numeric value is encountered
            print(f"Ignoring data point due to non-numeric value: {aircraft}, Field {e}")

    #return np.array(features), np.array(labels)
    features = np.array(features)
    labels = np.array(labels)

    # # Scale features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    return features, labels

def handle_alt_baro(alt_baro):
    if alt_baro == "ground":
        return 0.0
    else:
        return alt_baro
    
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(10,)),  # Define input shape explicitly
    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 192 (768.00 B)

In [99]:
spoofed_features, spoofed_labels = preprocess_data(aircraft_data_spoofed)
not_spoofed_features, not_spoofed_labels = preprocess_data(aircraft_data_valid)

# Combine the data
all_features = np.vstack([spoofed_features, not_spoofed_features])
all_labels = np.concatenate([spoofed_labels, not_spoofed_labels])

print(all_features, all_labels)

[[ 1.21698152  0.12338133  0.30270961 ...  1.3314093   0.79770954
   1.07453948]
 [ 0.41029097  0.62593615  0.55088812 ...  0.39182697 -1.23409194
  -0.62562485]
 [-1.15802799  1.48547818 -1.42499466 ... -1.5802634   0.72848291
   0.23125797]
 ...
 [-1.10993969 -1.32685152 -1.21581286 ... -1.17306362  0.07194092
  -1.81647978]
 [-1.10993969 -1.32685152 -1.21581286 ... -0.10443824  0.02871073
   0.06106721]
 [-1.10993969 -1.32685152 -1.21581286 ...  0.62940328 -0.11199928
   1.19022134]] [1 1 1 ... 0 0 0]


In [100]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

# Train the model !!! ADDED CLASS_WEIGHT, MIGHT NOT BE NEEDED
#class_weight={0: 1., 1: 10.}
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5215 - loss: 0.8745 - val_accuracy: 0.6735 - val_loss: 0.6385
Epoch 2/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5954 - loss: 0.7324 - val_accuracy: 0.7641 - val_loss: 0.5513
Epoch 3/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6540 - loss: 0.6413 - val_accuracy: 0.8158 - val_loss: 0.4995
Epoch 4/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7009 - loss: 0.5879 - val_accuracy: 0.8336 - val_loss: 0.4593
Epoch 5/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7221 - loss: 0.5607 - val_accuracy: 0.8496 - val_loss: 0.4260
Epoch 6/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.7548 - loss: 0.5138 - val_accuracy: 0.8640 - val_loss: 0.3954
Epoch 7/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7735 - loss: 0.4859 - val_accuracy: 0.8778 - val_loss: 0.3682
Epoch 8/20
337/337 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.7960 - loss: 0.4523 - val_accuracy

In [101]:
model_name = "TestmodelV4.keras"
model.save(model_name)
print(f"Model saved to {model_name}!")

Model saved to TestmodelV4.keras!


In [102]:
import json
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import folium

selected_file = "./samples/testing/customsample.json"

file = open(selected_file)
json_contents = json.load(file)
aircraft = json_contents["aircraft"]

planes = []

class PlaneLog:
    def __init__(self, hex):
        self.hex = hex
        self.planes = []
    def add_plane(self, plane_record):
        self.planes.append(plane_record)

for record in aircraft:
    matched_plane = False
    for plane in planes:
        if plane.hex == record["hex"]:
            plane.add_plane(record)
            matched_plane = True
    if matched_plane == False:
        new_plane = PlaneLog(record["hex"])
        new_plane.add_plane(record)
        planes.append(new_plane)
            
file.close()

print(f"🛫 Detected {len(planes)} planes!")
for i, plane in enumerate(planes):
    print(f"Plane #{i + 1}: {plane.hex} - {len(plane.planes)} records")

def find_plane(hex):
    for plane in planes:
        if plane.hex == hex:
            return plane
        
model = tf.keras.models.load_model("TestmodelV4.keras")
model.summary()
print(model.input_shape, model.output_shape)
def handle_alt_baro(alt_baro):
    if alt_baro == "ground":
        return 0.0
    else:
        return alt_baro

def predict_adsb_data(adsb_message):
    try:
        feature_vector = [
            handle_alt_baro(adsb_message.get('alt_baro', 0)),
            #float(adsb_message.get('alt_baro', 0)),
            float(adsb_message.get('gs', 0)),
            float(adsb_message.get('track', 0)),
            float(adsb_message.get('baro_rate', 0)),
            float(adsb_message.get('lat', 0)),
            float(adsb_message.get('lon', 0)),
            float(adsb_message.get('seen_pos', 0)),
            float(adsb_message.get('messages', 0)),
            float(adsb_message.get('seen', 0)),
            float(adsb_message.get('rssi', 0)),
        ]
    except ValueError as e:
        print(f"Error processing ADS-B message: {adsb_message}, Field: {e}")
        return None

    # scaler = StandardScaler()
    # features_scaled = scaler.fit_transform([feature_vector])
    # prediction = model.predict(np.array(features_scaled))

    prediction = model.predict(np.array([feature_vector]))
    print(prediction[:10])
    return prediction[0][0]
    #return int(round(prediction[0][0]))
    
from IPython.display import display
m = folium.Map(location = [51.5800, 5.1875], tiles ='OpenStreetMap', zoom_start=3, max_bounds=True, min_zoom=2)

#scaler = StandardScaler()

for plane in planes:
    #print(plane.hex)
    aircraft_data = find_plane(plane.hex)
    if aircraft_data == None:
        continue
        
    results = predict_adsb_data(aircraft_data.planes[0])
    #print(f'[RESULTS: {results}]')
    spoofed = (True if results > 0.5 else False)
    percentage = results * 100

    text = ("Spoofed" if spoofed else "Not Spoofed")
    #print(f"{text} -#-- {percentage}% prediction of being spoofed")
    
    plane_values = [{"key": "hex", "value": "Hex (24-bit ICAO identifier)"}, {"key": "type", "value": "Data Source"}, {"key": "flight", "value": "Flight Callsign"}, {"key": "r", "value": "Aircraft Registration"}, {"key": "t", "value": "Aircraft Type"}, {"key": "alt_baro", "value": "Aircraft Barometric Altitude (feet)"}, {"key": "gs", "value": "Ground Speed (knots)"}, {"key": "track", "value": "True Track (degrees)"}, {"key": "baro_rate", "value": "Rate of Change of Barometric Altitude (feet/minute)"}, {"key": "lat", "value": "Latitude"}, {"key": "lon", "value": "Longitude"}, {"key": "seen_pos", "value": "Updated Time Ago (seconds)"}, {"key": "messages", "value": "Total # of Mode S Messages"}, {"key": "seen", "value": "Last Timing Message (seconds)"}, {"key": "rssi", "value": "Signal Power (dbFs)"}]
    
    def find_value(id):
        for x in plane_values:
            if x["key"] == id:
                return x

    plane_data = aircraft_data.planes[0]
    plane_coordinates = (plane_data["lat"], plane_data["lon"])

    # Draw map markers
    folium.CircleMarker(
        location=plane_coordinates,
        radius=5,
        color=("red" if spoofed else "green"),
        fill=True,
        fill_color=("red" if spoofed else "green"),
        fill_opacity=0.7,
        popup='<br>Hex: ' + plane.hex + '<br>Lat: ' + str(aircraft_data.planes[0]['lat']) + '<br>Lon: ' + str(aircraft_data.planes[0]['lon'])
    ).add_to(m)

display(m)

🛫 Detected 105 planes!
Plane #1: abd6c6 - 1 records
Plane #2: 869560 - 1 records
Plane #3: a60b1e - 1 records
Plane #4: 86e79a - 1 records
Plane #5: 383E36 - 1 records
Plane #6: a4c79b - 1 records
Plane #7: 71c011 - 1 records
Plane #8: 78072a - 1 records
Plane #9: aa5bc5 - 1 records
Plane #10: aa8499 - 1 records
Plane #11: a1c7e4 - 1 records
Plane #12: adb4f0 - 1 records
Plane #13: a973be - 1 records
Plane #14: acebbe - 1 records
Plane #15: c02ec7 - 1 records
Plane #16: 86cf16 - 1 records
Plane #17: 86e40e - 1 records
Plane #18: a54847 - 1 records
Plane #19: 71c393 - 1 records
Plane #20: 71be33 - 1 records
Plane #21: a1b157 - 1 records
Plane #22: a828e1 - 1 records
Plane #23: a4c74c - 1 records
Plane #24: adc50d - 1 records
Plane #25: aae22e - 1 records
Plane #26: a97128 - 1 records
Plane #27: a4db4e - 1 records
Plane #28: a2f2fa - 1 records
Plane #29: 8990ee - 1 records
Plane #30: 872f7e - 1 records
Plane #31: a96412 - 1 records
Plane #32: a980b9 - 1 records
Plane #33: a581c8 - 1 reco

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,221 (36.02 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 6,020 (23.52 KB)

(None, 10) (None, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/s